# Evaluating Steerability of Different Concepts

In this notebook, we will evaluate steerability across several concepts of interest. We consider two types of concepts: 
(i) Abstract concepts, e.g. 'truthfulness', 'power-seeking-inclination' 
(ii) Linguistic concepts, e.g. 'verb->verb-able', 'adjective->adjective-ly' 

In [ ]:
from repepo.steering.sweeps.constants import (
    ALL_ABSTRACT_CONCEPT_DATASETS,
    ALL_TOKEN_CONCEPT_DATASETS, 
    ALL_LANGUAGES,
    ALL_LLAMA_7B_LAYERS,
    ALL_MULTIPLIERS
)

from repepo.steering.sweeps.configs import (
    get_abstract_concept_config,
    get_token_concept_config
)

from repepo.steering.run_sweep import (
    run_sweep, 
    load_sweep_results
)

from repepo.steering.plots.utils import (
    get_config_fields,
    make_results_df
)

In [ ]:
# Define the sweep to run over. 

from itertools import product

abstract_setting = {
    # "datasets": ALL_ABSTRACT_CONCEPT_DATASETS,
    "datasets": [
        "anti-immigration",
        "conscientiousness",
        "self-replication",
        "corrigible-neutral-HHH",
        "myopic-reward",
        "power-seeking-inclination",
        "sycophancy_train",
        "truthfulqa",
    ],
    "layers": [13],
    "multipliers": [-1.0, 0.0, 1.0]
}

linguistic_setting = {
    "datasets": [
        "D03 [adj+ly_reg]",
        "D07 [verb+able_reg]",
        "I06 [verb_inf - Ving]",
    ],
    "layers": [13],
    "multipliers": [-1.0, 0.0, 1.0]
}

def iter_config():
    for dataset, layer, multiplier in product(
        abstract_setting["datasets"], 
        abstract_setting["layers"], 
        abstract_setting["multipliers"]
    ):
        yield get_abstract_concept_config(
            dataset=dataset,
            layer=layer,
            multiplier=multiplier
        )

    for dataset, layer, multiplier in product(
        linguistic_setting["datasets"], 
        linguistic_setting["layers"], 
        linguistic_setting["multipliers"]
    ):
        yield get_token_concept_config(
            dataset=dataset,
            layer=layer,
            multiplier=multiplier
        )


In [ ]:
# Optionally, run the sweep and load results. 
# If sweep was already run, set RUN = False.
RUN = False

configs = list(iter_config())
if RUN:
    run_sweep(configs)

results = load_sweep_results(configs)

In [ ]:
# Construct a DataFrame from the results.
df = make_results_df(results)
print(len(df))
df.head()

In [ ]:
# Plot the change in positive prob and negative prob for one example. 

import seaborn as sns 
import matplotlib.pyplot as plt

def plot(df):
    example = df.iloc[0]
    df = df[df["test_positive_example.text"] == example["test_positive_example.text"]]
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    # Plot positive token logit, negative token logit.
    sns.lineplot(data=df, x="multiplier", y="test_positive_token.logprob", label="Positive logprob", ax=ax)
    sns.lineplot(data=df, x="multiplier", y="test_negative_token.logprob", label="Negative logprob", ax=ax)

plot(df)

In [ ]:
# Plot the change in positive token logit and negative token logit for one example. 

import seaborn as sns 
import matplotlib.pyplot as plt

def plot(df):
    example = df.iloc[0]
    df = df[df["test_positive_example.text"] == example["test_positive_example.text"]]
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    # Plot positive token logit, negative token logit.
    sns.lineplot(data=df, x="multiplier", y="test_positive_token.logit", label="Positive logit", ax=ax)
    sns.lineplot(data=df, x="multiplier", y="test_negative_token.logit", label="Negative logit", ax=ax)
    # Also plot the logit_mean
    # sns.lineplot(data=df, x="multiplier", y="test_positive_token.logit_mean", label="Logit mean", ax=ax)
    # sns.lineplot(data=df, x="multiplier", y="test_negative_token.logit_mean", label="Logit mean", ax=ax)

plot(df)